In [ ]:
# baseline accuracy 
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
927,"One thing about Hollywood, someone has a succe...",positive
382,OK. Who brought the cheese. I love it. During ...,positive
961,I don't see why all the people are giving this...,positive
423,A very tired looking Burt Reynolds plays a mer...,negative
724,Tom is listening to one of those old-time radi...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
927,one thing hollywood someone success always rus...,positive
382,ok brought cheese love it run became phenomeno...,positive
961,see people giving film negative review loved m...,positive
423,tired looking burt reynolds play mercenary bat...,negative
724,tom listening one old time radio broadcast som...,negative


In [5]:
df['sentiment'].value_counts()


sentiment
negative    254
positive    246
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
927,one thing hollywood someone success always rus...,1
382,ok brought cheese love it run became phenomeno...,1
961,see people giving film negative review loved m...,1
423,tired looking burt reynolds play mercenary bat...,0
724,tom listening one old time radio broadcast som...,0


In [8]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/sayyedsabirali/capstone-project.mlflow')
dagshub.init(repo_owner='sayyedsabirali', repo_name='capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Accessing as sayyedsabirali

Initialized MLflow to track repo "sayyedsabirali/capstone-project"

Repository sayyedsabirali/capstone-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/ab25fa43769046e5b1b26e0b6789054a', creation_time=1755346030661, experiment_id='0', last_update_time=1755346030661, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [11]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-08-16 21:22:38,513 - INFO - Starting MLflow run...
2025-08-16 21:22:39,165 - INFO - Logging preprocessing parameters...
2025-08-16 21:22:40,317 - INFO - Initializing Logistic Regression model...
2025-08-16 21:22:40,317 - INFO - Fitting the model...
2025-08-16 21:22:40,350 - INFO - Model training complete.
2025-08-16 21:22:40,351 - INFO - Logging model parameters...
2025-08-16 21:22:40,707 - INFO - Making predictions...
2025-08-16 21:22:40,707 - INFO - Calculating evaluation metrics...
2025-08-16 21:22:40,721 - INFO - Logging evaluation metrics...
2025-08-16 21:22:45,468 - INFO - Model training and logging completed in 6.30 seconds.
2025-08-16 21:22:45,476 - INFO - Accuracy: 0.664
2025-08-16 21:22:45,476 - INFO - Precision: 0.6935483870967742
2025-08-16 21:22:45,476 - INFO - Recall: 0.6515151515151515
2025-08-16 21:22:45,476 - INFO - F1 Score: 0.671875


🏃 View run awesome-dolphin-56 at: https://dagshub.com/sayyedsabirali/capstone-project.mlflow/#/experiments/0/runs/c1affd716e5043cbb6eb22a391eba9c3
🧪 View experiment at: https://dagshub.com/sayyedsabirali/capstone-project.mlflow/#/experiments/0
